In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import friedmanchisquare
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from itertools import combinations
import openpyxl
import scienceplots

In [2]:
#Load Questionairs (in flight)
# se base foler

# Define the base folder path
#base_folder = r'C:\Users\ejnar\Desktop\Special sync\Special'
base_folder = r'C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Special'
# Define the path to the Data folder
data_folder = os.path.join(base_folder, 'Data')

# Define the path to the Excel file
excel_file_path1 = os.path.join(data_folder, 'In-Flight-Hepa-7.xlsx')
excel_file_path2 = os.path.join(data_folder, 'In-Flight-Hepa-5.xlsx')
excel_file_path3 = os.path.join(data_folder, 'In-Flight-APS-U.xlsx')
excel_file_path4 = os.path.join(data_folder, 'In-Flight-APS-N.xlsx')

# Load the Excel file into a DataFrame
dfHepa7 = pd.read_excel(excel_file_path1)
dfHepa5 = pd.read_excel(excel_file_path2)
dfAPSU = pd.read_excel(excel_file_path3)
dfAPSN = pd.read_excel(excel_file_path4)

#dataframe
dataframe = [dfHepa7, dfHepa5, dfAPSN, dfAPSU,]  # Raw

In [3]:
#change name surfix and nametag
#nametag on boxplot
#manualt change

#nametag change
#nametag = "All"
#nametag = "Men"
#nametag = "Women"
#nametag = "Young"
#nametag = "Juniors"
#nametag = "Elderly"
#nametag = "Seniors"
#nametag = "Sensitive"
nametag = "Non-Sensitive"
name_suffix = nametag
#output folder
output_folder = os.path.join(os.getcwd(), 'Figur', name_suffix, name_suffix)

In [4]:
#Dictionary to map old column names to new ones
columns_mapping = {
    'Right now the air feels too: (Fresh)': 'Right Now Air Feels: Stuffy / Fresh',
    'Right now I feel as follows:(irritation nose)': 'RNIFAF: Irritation in Nose / No Irritation',
    'Right now I feel as follows:(irritation throat)': 'RNIFAF: Irritation in Throat / No Irritation',
    'Right now I feel as follows:(irritation eyes)': 'RNIFAF: Irritation in Eyes / No Irritation',
    'Right now the air feels too:(Warm)': 'Right Now Air Feels: Cold / Warm',
    'Right now the air feels too:(humid)': 'Right Now Air Feels: Dry / Humid',
    'How do you assess the movement of air in the cabin?': 'Air Movement: Drafty / Still',
    'Right now my surroundings feel too:(bright)': 'Surroundings Feel: Dark / Bright',
    'Right now my surroundings feel too:(quite)': 'Surroundings Feel: Noisy / Quiet',
    'How would you assess the overall environment in the cabin?': 'Cabin Environment: Unacceptable / Acceptable',
    'Right now I feel as follows:(nose dry)': 'RNIFAF: Nose Running / Dry',
    'Right now I feel as follows:(nose clear)': 'RNIFAF: Nose Blocked / Clear',
    'Right now I feel as follows:(throat dry)': 'RNIFAF: Throat Dry / Not Dry',
    'Right now I feel as follows:(mouth dry)': 'RNIFAF: Mouth Dry / Not Dry',
    'Right now I feel as follows:(lips dry)': 'RNIFAF: Lips Dry / Not Dry',
    'Right now I feel as follows:(skin dry)': 'RNIFAF: Skin Dry / Not Dry',
    'Right now I feel as follows:(eyes dry)': 'RNIFAF: Eyes Dry / Not Dry',
    'Right now I feel as follows:(headache)': 'RNIFAF: Headache / No Headache',
    'Right now I feel as follows:(thirsty)': 'RNIFAF: Thirsty / Not Thirsty',
    'Right now I feel as follows:(concentrating)': 'RNIFAF: Difficulty Concentrating / No Difficulty',
    'Right now I feel as follows:(rested)': 'RNIFAF: Tired / Rested',
    'Right now I feel as follows:(dizzy)': 'RNIFAF: Dizzy / Not Dizzy',
    'Right now I feel as follows:(alert)': 'RNIFAF: Sleepy / Alert',
    'Right now I feel as follows:(relaxed)': 'RNIFAF: Anxious / Relaxed',
    'Right now I feel as follows:(bored)': 'RNIFAF: Bored / Interested',
    'Right now I feel as follows:(calm)': 'RNIFAF: Scared / Calm',
    'How would you assess the smell in the cabin?': 'Smell in Cabin: Odour / No Odour',
    'How would you assess the overall quality of air in the cabin?': 'Air Quality: Unacceptable / Acceptable',
    'Right now I feel as follows:(claustrophobic)': 'RNIFAF: Claustrophobic / Not Claustrophobic'
}

# Apply the renaming
dfHepa7 = dfHepa7.rename(columns=columns_mapping) 
dfHepa5 = dfHepa5.rename(columns=columns_mapping) 
dfAPSU = dfAPSU.rename(columns=columns_mapping) 
dfAPSN = dfAPSN.rename(columns=columns_mapping) 

In [5]:
# Removing columns
columns_to_remove = [
    'Please enter your seat number (this can be seen on your boarding pass):','Which survey are you answering right now?',	
    'Language', 'E-mail', 'Overall Status - New', 'Overall Status - Distributed',
    'Overall Status - Partially Complete', 'Overall Status - Complete',
    'Overall Status - Rejected',
]

dfHepa7 = dfHepa7.drop(columns=columns_to_remove)
dfHepa5 = dfHepa5.drop(columns=columns_to_remove)
dfAPSU = dfAPSU.drop(columns=columns_to_remove)
dfAPSN = dfAPSN.drop(columns=columns_to_remove)

In [6]:
# Removing all der ikke har været der alle 4 gange
# List of participant numbers to keep
participants_to_keep =[1, 2, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 
                        30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 
                        57, 58, 59, 61, 63, 64, 65, 66, 67, 68, 69]

# Filter the DataFrame
dfHepa7 = dfHepa7[dfHepa7['Please enter your participant number (this can be seen on your boarding pass):'].isin(participants_to_keep)]
dfHepa5 = dfHepa5[dfHepa5['Please enter your participant number (this can be seen on your boarding pass):'].isin(participants_to_keep)]
dfAPSU = dfAPSU[dfAPSU['Please enter your participant number (this can be seen on your boarding pass):'].isin(participants_to_keep)]
dfAPSN = dfAPSN[dfAPSN['Please enter your participant number (this can be seen on your boarding pass):'].isin(participants_to_keep)]

In [7]:
#person filter
Participant_number_men = [1, 3, 7, 10, 11, 12, 16, 18, 19, 22, 24, 26, 27, 30, 31, 32, 33, 34, 37, 39, 42, 44, 46, 47, 49, 50, 51, 53, 55, 59, 61, 62, 63, 64, 65, 67, 68]

Participant_number_women = [2, 4, 5, 6, 8, 9, 13, 14, 15, 17, 20, 21, 23, 25, 28, 29, 35, 36, 38, 40, 41, 43, 45, 52, 54, 56, 57, 58, 60, 66, 69]

Participant_number_young = [1, 3, 4, 5, 6, 9, 13, 14, 15, 16, 18, 19, 20, 21, 26, 29, 30, 32, 33, 34, 37, 38, 39, 43, 45, 46, 47, 49, 51, 52, 55, 57, 59, 60, 61, 62, 65, 67, 68]

Participant_number_old = [ 2, 7, 8, 10, 11, 12, 17, 22, 23, 24, 25, 27, 28, 31, 35, 36, 40, 41, 42, 44, 50, 53, 54, 56, 58, 63, 64, 66, 69]

Participant_number_sen = [34, 35, 65, 39, 19, 4, 22, 6, 7, 9, 47, 10, 31, 32, 15, 17, 37]

Participant_number_nonsen = [1, 52, 2, 36, 18, 53, 38, 20, 54, 21, 56, 41, 23, 55, 57, 58, 24, 42, 26, 59, 44, 27, 8, 69, 28, 29, 30, 61, 45, 46, 11, 63, 64, 49, 13, 14, 66, 16, 51, 67, 50, 68]

# Filter the DataFrame
dfHepa7 = dfHepa7[dfHepa7['Please enter your participant number (this can be seen on your boarding pass):'].isin(Participant_number_nonsen)]
dfHepa5 = dfHepa5[dfHepa5['Please enter your participant number (this can be seen on your boarding pass):'].isin(Participant_number_nonsen)]
dfAPSU = dfAPSU[dfAPSU['Please enter your participant number (this can be seen on your boarding pass):'].isin(Participant_number_nonsen)]
dfAPSN = dfAPSN[dfAPSN['Please enter your participant number (this can be seen on your boarding pass):'].isin(Participant_number_nonsen)]

In [8]:
#Boxplot of questionair without legend box
def create_combined_boxplots(dataframes, labels, output_folder, colors=None, name_suffix="", nametag="", y_min=0, y_max=102):
    """Create combined box plots for a specified list of DataFrames."""
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Apply the 'science' style and 'no-latex'
    plt.style.use(['science', 'no-latex'])

    # Apply the custom color palette
    custom_palette = ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33']
    if colors is None:
        colors = custom_palette
    sns.set_palette(custom_palette)

    # Assuming all DataFrames have the same sort orders
    sort_orders = dataframes[0]['Sort Order'].unique()

    # Define your custom x-tick labels
    custom_labels = ["Q1", "Q2", "Q3", "Q4"]

    # Loop through each column in the DataFrames
    for column in dataframes[0].columns:
        if pd.api.types.is_numeric_dtype(dataframes[0][column]) and column != 'Sort Order':  # Exclude 'Sort Order' column
            plt.figure(figsize=(10, 7))  # Updated figure size

            # Prepare combined data for box plots
            combined_data = pd.concat([
                dataframe.assign(Dataset=labels[i]) 
                for i, dataframe in enumerate(dataframes)
            ])
            combined_data = combined_data[combined_data['Sort Order'] != 5] 

            # Create boxplot
            sns.boxplot(
                x='Sort Order', y=column, hue='Dataset', data=combined_data, 
                palette=custom_palette, order=sorted(combined_data['Sort Order'].unique())
            )

            # Add grid
            plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.75, axis='y')

            # Add titles and labels 
            plt.title(f'{column} - {nametag}', fontsize=20)
            plt.xlabel('Questionnaire', fontsize=18)

            # Automatically format the y-axis label 
            if ':' in column and ' / ' in column:
                label, values = column.split(':')
                left, right = values.strip().split(' / ')
                plt.ylabel(
                    f'$\\it{{{left.strip().replace(" ", "\\ " )}}}$ ---- Rating [-] ---- $\\it{{{right.strip().replace(" ", "\\ " )}}}$',
                    fontsize=18
                )
            else:
                plt.ylabel(
                    '$\\it{None}$ ---- Rating [-] ---- $\\it{Overpowering}$',
                    fontsize=18
                )

            # Set y-axis limits
            plt.ylim(y_min, y_max)

            # Set custom x-ticks to match your original setup
            plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=16)

            # Set y-ticks with the same font size as x-ticks
            plt.yticks(fontsize=16)

            # Remove the legend
            plt.legend().set_visible(False)

            # Adjust layout 
            plt.tight_layout()

            # Save the plot
            plot_filename = f"{sanitize_filename(column)}_{sanitize_filename(name_suffix)}.png"
            plt.savefig(os.path.join(output_folder, plot_filename), bbox_inches='tight')
            plt.close()  # Close the plot to free memory


In [9]:
#Boxplot of questionair with legend box
def sanitize_filename(name):
    """Replace invalid characters in a filename with underscores."""
    return re.sub(r'[<>:"/\\|?*]', '_', name)

def create_combined_boxplots(dataframes, labels, output_folder, colors=None, name_suffix="", nametag="", y_min=0, y_max=102):
    """Create combined box plots for a specified list of DataFrames."""
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Apply the 'science' style and 'no-latex'
    plt.style.use(['science', 'no-latex'])

    # Apply the custom color palette
    custom_palette = ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33']
    if colors is None:
        colors = custom_palette
    sns.set_palette(custom_palette)

    # Assuming all DataFrames have the same sort orders
    sort_orders = dataframes[0]['Sort Order'].unique()

    # Define your custom x-tick labels
    custom_labels = ["Q1", "Q2", "Q3", "Q4"]

    # Loop through each column in the DataFrames
    for column in dataframes[0].columns:
        if pd.api.types.is_numeric_dtype(dataframes[0][column]) and column != 'Sort Order':  # Exclude 'Sort Order' column
            plt.figure(figsize=(10, 7))  # Updated figure size

            # Prepare combined data for box plots, filtering out Sort Order = 5
            combined_data = pd.concat([
                dataframe.assign(Dataset=labels[i]) 
                for i, dataframe in enumerate(dataframes)
            ])
            combined_data = combined_data[combined_data['Sort Order'] != 5]  # Exclude Sort Order = 5

            # Create boxplot
            sns.boxplot(
                x='Sort Order', y=column, hue='Dataset', data=combined_data, 
                palette=custom_palette, order=sorted(combined_data['Sort Order'].unique())
            )

            # Add grid (horizontal lines only)
            plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.75, axis='y')

            # Add titles and labels without changing their styling
            plt.title(f'{column} - {nametag}', fontsize=20)
            plt.xlabel('Questionnaire', fontsize=18)

            # Automatically format the y-axis label 
            if ':' in column and ' / ' in column:
                label, values = column.split(':')
                left, right = values.strip().split(' / ')
            # Add \ to ensure spaces are preserved 
                plt.ylabel(
                f'$\\it{{{left.strip().replace(" ", "\\ " )}}}$ ---- Rating [-] ---- $\\it{{{right.strip().replace(" ", "\\ " )}}}$',
                fontsize=18
                    )
            else:
                plt.ylabel(
                '$\\it{None}$ ---- Rating [-] ---- $\\it{Overpowering}$',
                fontsize=18
                )

            # Set y-axis limits
            plt.ylim(y_min, y_max)

            # Set custom x-ticks
            plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=16)

            # Set y-ticks with the same font size as x-ticks
            plt.yticks(fontsize=16)

            # Add legend with similar style as the second code
            plt.legend(
                title="Conditions", title_fontsize=16, fontsize=14, loc='center left', 
                bbox_to_anchor=(1, 0.5), frameon=False  # Remove the box around the legend
            )

            # Adjust layout to ensure no overlap
            plt.tight_layout()

            # Save the plot
            plot_filename = f"{sanitize_filename(column)}_{sanitize_filename(name_suffix)}.png"
            plt.savefig(os.path.join(output_folder, plot_filename), bbox_inches='tight')
            plt.close()  


In [10]:
#Making dataframes
#first split the first two queistions as their scale is diffrent 
# Create a new DataFrame for "How would you assess the overall quality of air in the cabin?"
df_quality_of_air_Hepa7 = dfHepa7[[
                        "Sort Order","Cabin Environment: Unacceptable / Acceptable", 
                        "Air Quality: Unacceptable / Acceptable"]].copy()
df_quality_of_air_Hepa5 = dfHepa5[[
                        "Sort Order","Cabin Environment: Unacceptable / Acceptable", 
                        "Air Quality: Unacceptable / Acceptable"]].copy()
df_quality_of_air_APSU = dfAPSU[[
                        "Sort Order", "Cabin Environment: Unacceptable / Acceptable",
                        "Air Quality: Unacceptable / Acceptable"]].copy()
df_quality_of_air_APSN = dfAPSN[[
                        "Sort Order","Cabin Environment: Unacceptable / Acceptable", 
                        "Air Quality: Unacceptable / Acceptable"]].copy()

# Create a new DataFrame for "How would you assess the smell in the cabin?"
df_smell_in_cabin_Hepa7 = dfHepa7[[
                        "Sort Order", 
                        "Smell in Cabin: Odour / No Odour"]].copy()
df_smell_in_cabin_Hepa5 = dfHepa5[[ 
                        "Sort Order", 
                        "Smell in Cabin: Odour / No Odour"]].copy()
df_smell_in_cabin_APSU = dfAPSU[[ 
                        "Sort Order", 
                        "Smell in Cabin: Odour / No Odour"]].copy()
df_smell_in_cabin_APSN = dfAPSN[[ 
                        "Sort Order", 
                        "Smell in Cabin: Odour / No Odour"]].copy()


# Drop the two columns from the original DataFrame
dfHepa71 = dfHepa7.drop(columns=["Air Quality: Unacceptable / Acceptable","Cabin Environment: Unacceptable / Acceptable", 
                              "Smell in Cabin: Odour / No Odour","Please enter your participant number (this can be seen on your boarding pass):"])
dfHepa51 = dfHepa5.drop(columns=["Air Quality: Unacceptable / Acceptable", "Cabin Environment: Unacceptable / Acceptable",
                              "Smell in Cabin: Odour / No Odour","Please enter your participant number (this can be seen on your boarding pass):"])
dfAPSU1 = dfAPSU.drop(columns=["Air Quality: Unacceptable / Acceptable", "Cabin Environment: Unacceptable / Acceptable",
                              "Smell in Cabin: Odour / No Odour","Please enter your participant number (this can be seen on your boarding pass):"])
dfAPSN1 = dfAPSN.drop(columns=["Air Quality: Unacceptable / Acceptable", "Cabin Environment: Unacceptable / Acceptable",
                              "Smell in Cabin: Odour / No Odour","Please enter your participant number (this can be seen on your boarding pass):"])

In [11]:
#først plot som er raw data 
# Raw data direkte fra quesionair
dataframesRaw100 = [dfHepa71, dfHepa51, dfAPSN1, dfAPSU1,]  # Raw

dataframesRawAir = [df_quality_of_air_Hepa7, df_quality_of_air_Hepa5, df_quality_of_air_APSN, df_quality_of_air_APSU]  # Raw Air

dataframesRawSmell = [df_smell_in_cabin_Hepa7, df_smell_in_cabin_Hepa5, df_smell_in_cabin_APSN, df_smell_in_cabin_APSU]  # Raw Smell

colors = ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33'] # colors
labels = ['HEPA 7', 'HEPA 5', 'APS N', 'APS U']  # Labels for each DataFrame

# Create the combined boxplots with a name suffix and custom colors
create_combined_boxplots(dataframesRaw100, labels, output_folder, colors=colors, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=102)
create_combined_boxplots(dataframesRawAir, labels, output_folder, colors=colors, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=11)
create_combined_boxplots(dataframesRawSmell, labels, output_folder, colors=colors, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=17)

In [12]:
#Friedman willcoxon by questionair
# Define output name
output_path = f'Freidman-Willcoxon-by-Questinair-{name_suffix}.xlsx'  # Set output file path

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}
# Specify columns not to test
columns_not_to_test = ['Please enter your participant number (this can be seen on your boarding pass):', 'Sort Order']

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for dataset_name, df in dataframes.items():
        # Determine columns to test by excluding the specified ones
        columns_to_test = df.columns.difference(columns_not_to_test).tolist()

        # Initialize DataFrames to store results for the current dataset
        friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant"])
        wilcoxon_results_list = []

        # Loop through each column
        for col in columns_to_test:
            # Filter out data for each "Sort Order" value (1 to 4) for the Friedman test
            sort_order_groups = [df[df["Sort Order"] == i][col].dropna().values for i in range(1, 5)]
            
            # Perform the Friedman test
            friedman_stat, friedman_p = friedmanchisquare(*sort_order_groups)
            
            # Determine if the result is significant
            is_significant = 1 if friedman_p < alpha else 0
            
            # Append Friedman test results to the DataFrame
            temp_df = pd.DataFrame({
                "Column": [col],
                "Statistic": [friedman_stat],
                "p-value": [friedman_p],
                "Alpha": [alpha],
                "Significant": [is_significant]
            })
            friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)

            # If significant, perform pairwise Wilcoxon tests
            if is_significant:
                # Get all pairs of sort order values to test (1 to 4)
                pairs = combinations(range(1, 5), 2)
                for (i, j) in pairs:
                    # Get data for each pair
                    group1 = df[df["Sort Order"] == i][col].dropna().values
                    group2 = df[df["Sort Order"] == j][col].dropna().values

                    # Calculate average values for comparison
                    avg_group1 = group1.mean() if len(group1) > 0 else None
                    avg_group2 = group2.mean() if len(group2) > 0 else None

                    # Determine the average comparison value
                    if avg_group1 is not None and avg_group2 is not None:
                        if avg_group1 > avg_group2:
                            avg_comparison = 1
                        elif avg_group1 < avg_group2:
                            avg_comparison = -1
                        else:
                            avg_comparison = 0
                    else:
                        avg_comparison = None  # Not enough data to calculate

                    # Calculate median values for comparison
                    med_group1 = pd.Series(group1).median() if len(group1) > 0 else None
                    med_group2 = pd.Series(group2).median() if len(group2) > 0 else None

                    # Determine the median comparison value
                    if med_group1 is not None and med_group2 is not None:
                        if med_group1 > med_group2:
                            med_comparison = 1
                        elif med_group1 < med_group2:
                            med_comparison = -1
                        else:
                            med_comparison = 0
                    else:
                        med_comparison = None  

                    # Ensure both groups have data for the Wilcoxon test
                    if len(group1) > 0 and len(group2) > 0:
                        wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                        wilcox_significant = 1 if wilcox_p < alpha else 0  
                        
                        # Append results to the Wilcoxon results list
                        wilcoxon_results_list.append({
                            "Column": col,
                            "Pair": f"{i}-{j}",
                            "Statistic": wilcox_stat,
                            "p-value": wilcox_p,
                            "Alpha": alpha,
                            "Significant": wilcox_significant,
                            "Average": avg_comparison, 
                            "Median": med_comparison   
                        })

        # Save results for the current dataset in separate sheets
        friedman_results_df.to_excel(writer, sheet_name=f"{dataset_name} Friedman Results", index=False)
        
        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"{dataset_name} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.")

C:\Users\ejnar\AppData\Local\Temp\ipykernel_2304\3742034397.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_2304\3742034397.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_2304\3742034397.py:47: FutureWarning: The behavior of DataFrame 

Friedman and Wilcoxon test results saved to Freidman-Willcoxon-by-Questinair-Non-Sensitive.xlsx.


In [13]:
#Friedman willcoxon by conditions
# Define output name
output_path = f'Freidman-Willcoxon-by-Filter-{name_suffix}.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Specify columns not to test
columns_not_to_test = ['Please enter your participant number (this can be seen on your boarding pass):']

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # Loop through each "Sort Order" value (1 to 4)
    for sort_order in range(1, 5):
        # Results storage for Friedman and Wilcoxon tests
        friedman_results = []
        wilcoxon_results_list = []

        # Filter DataFrames by Sort Order
        filtered_dfs = {name: df[df["Sort Order"] == sort_order] for name, df in dataframes.items()}

        # Loop through each column 
        columns_to_test = list(set(dfHepa7.columns).difference(columns_not_to_test))
        for col in columns_to_test:
            # Collect data for the Friedman test
            groups = [df[col].dropna().values for df in filtered_dfs.values()]
            if all(len(group) > 1 for group in groups):  # Ensure all groups have data
                # Perform the Friedman test
                friedman_stat, friedman_p = friedmanchisquare(*groups)
                friedman_significant = 1 if friedman_p < alpha else 0

                # Save Friedman test results
                friedman_results.append({
                    "Sort Order": sort_order,
                    "Column": col,
                    "Statistic": friedman_stat,
                    "p-value": friedman_p,
                    "Alpha": alpha,
                    "Significant": friedman_significant
                })

                # Only perform Wilcoxon tests if the Friedman test is significant
                if friedman_significant:
                    dataset_pairs = list(combinations(filtered_dfs.items(), 2))
                    for (dataset1_name, df1), (dataset2_name, df2) in dataset_pairs:
                        group1 = df1[col].dropna().values
                        group2 = df2[col].dropna().values

                        # Calculate average values for comparison
                        avg_group1 = group1.mean() if len(group1) > 0 else None
                        avg_group2 = group2.mean() if len(group2) > 0 else None

                        avg_comparison = None
                        if avg_group1 is not None and avg_group2 is not None:
                            avg_comparison = (
                                1 if avg_group1 > avg_group2 else -1 if avg_group1 < avg_group2 else 0
                            )

                        # Calculate median values for comparison
                        med_group1 = pd.Series(group1).median() if len(group1) > 0 else None
                        med_group2 = pd.Series(group2).median() if len(group2) > 0 else None

                        med_comparison = None
                        if med_group1 is not None and med_group2 is not None:
                            med_comparison = (
                                1 if med_group1 > med_group2 else -1 if med_group1 < med_group2 else 0
                            )

                        # Ensure groups have data for Wilcoxon test
                        if len(group1) > 0 and len(group2) > 0:
                            wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                            wilcox_significant = 1 if wilcox_p < alpha else 0

                            # Save Wilcoxon test results
                            wilcoxon_results_list.append({
                                "Sort Order": sort_order,
                                "Column": col,
                                "Dataset Pair": f"{dataset1_name} vs {dataset2_name}",
                                "Statistic": wilcox_stat,
                                "p-value": wilcox_p,
                                "Alpha": alpha,
                                "Significant": wilcox_significant,
                                "Average": avg_comparison,  
                                "Median": med_comparison  
                            })

        # Save Friedman results to a sheet
        if friedman_results:
            friedman_df = pd.DataFrame(friedman_results)
            friedman_df.to_excel(writer, sheet_name=f"Sort Order {sort_order} Friedman", index=False)

        # Save Wilcoxon results to a sheet
        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"Sort Order {sort_order} Wilcoxon", index=False)

print(f"Friedman and Wilcoxon test results with averages and medians saved to {output_path}.")


C:\Users\ejnar\OneDrive\DTU\speciale_projekt\venv\Lib\site-packages\scipy\stats\_stats_py.py:9064: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c
C:\Users\ejnar\OneDrive\DTU\speciale_projekt\venv\Lib\site-packages\scipy\stats\_stats_py.py:9064: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c
C:\Users\ejnar\OneDrive\DTU\speciale_projekt\venv\Lib\site-packages\scipy\stats\_stats_py.py:9064: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c
C:\Users\ejnar\OneDrive\DTU\speciale_projekt\venv\Lib\site-packages\scipy\stats\_stats_py.py:9064: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c


Friedman and Wilcoxon test results with averages and medians saved to Freidman-Willcoxon-by-Filter-Non-Sensitive.xlsx.


In [14]:
#Saving dataframe of Conditions
filename = f"Questionair_output_{nametag}.xlsx"

# Get the current working directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, filename)


# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Initialize an Excel writer
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Loop through each dataset and save it to a separate sheet
    for dataset_name, df in dataframes.items():
        # Save each DataFrame to its corresponding sheet
        df.to_excel(writer, sheet_name=dataset_name, index=False)

print(f"DataFrames saved to {file_path}")

DataFrames saved to C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Special\Code\Questionair_output_Non-Sensitive.xlsx
